# Data Cleaning and EDA


**Overview:** This notebook goes through data cleaning and exploratory data analysis in order to prepare for model building.

Specifically, we'll be walking through:

1. **Getting the data:** In this case, the data is stored in an SQL table
2. **Exploratory Data Analysis (EDA):** Visualizing the data to understand the distribution and correlation of different features.
3. **Data Cleaning:** Basic pre-processing techniques

The output of this notebook will be clean, organized data stored as a serialized (pickled) dataframe.

## Problem Statement
Build a predictive model that assigns default probabilities to loan applications and it is similar to what our credit modeling team is doing. Predict the probability that a customer is going to default (target=1) or not (target=0).

**Assumption:** The dataset provided contains credit and loan information about users and whether they have defaulted or not.

In [ ]:
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pickle

## Data Gathering

In [ ]:
# Connect to database
conn = psycopg2.connect("dbname=sample user=hiring password=g~+oc2ABz#-GE+u%pQ3Z host=anyfin-work-sample.cn2fqd2vd4bc.eu-west-1.rds.amazonaws.com")

# Activation connection cursor
#cur = conn.cursor()

# Run query and display
#cur.execute("SELECT * FROM creditdataset")
#rows = cur.fetchall()
#rows

df = pd.read_sql("SELECT * FROM creditdataset", conn)

# Close connections
#cur.close()
conn.close()

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
print(df.columns.values)

**Note:** Since ultimately, the goal will be to pick the best features to build the predictive model, it is important to learn as much as possible about the data. In my case, I don't have the time, or resources to learn what each one of the columns truly represent. I will limit my exploration of the data to the columns that make the most sense and proceed with my prediction. 

In [ ]:
df.describe()

## Exploratory Data Analysis (EDA)

In [ ]:
df.isna().sum().sort_values(ascending=False)

In [ ]:
df.nunique().sort_values(ascending=True)

In [ ]:
loan_type = df["loan_type"].value_counts()

plt.figure(figsize=(16,10))
sns.barplot(loan_type.index, loan_type.values)
plt.title('Distribution of Loan Types')
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('Type', fontsize=12)
plt.show()

**Observations:** Most popular loan type is "pos"

In [ ]:
big_city = df["big_city"].value_counts()

plt.figure(figsize=(16,10))
sns.barplot(big_city.index, big_city.values)
plt.title('Distribution of people who live in Big City')
plt.ylabel('Number of People', fontsize=12)
plt.xlabel('Big City', fontsize=12)
plt.show()

**Observations:** 

In [ ]:
target = df["target"].map({0:"No Default", 1:"Default"}).value_counts()

plt.figure(figsize=(16,10))
sns.barplot(target.index, target.values)
plt.title('Dafault vs No Default')
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('Type', fontsize=12)
plt.show()

**Observations:** As we can see, most people on the dataset don't default, this means that the data is unbalanced and training on this dataset might not give us the best predictive model

In [ ]:
spouse = df["target"].map({0:"Single", 1:"Married"}).value_counts()

plt.figure(figsize=(16,10))
sns.barplot(spouse.index, spouse.values)
plt.title('Distribution of Married vs Single clients')
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('Type', fontsize=12)
plt.show()

**Obervations:** The majority of people who applied for a loan are not married (no spouse). It would be interesting to see its correlation with the "external_score"

**Top 5 columns of interest:**
These are some of the columns that could have useful information to make a prediction. The goal is to explore them first, and then focus on the rest of the columns.

1. target
2. loan_type
3. income_gross
4. customer_age
5. credit_approved

**creditdataset:** This data set contains

In [ ]:
plt.figure(figsize=(16,10))
sns.swarmplot(x="has_spouce", y="external_score", data=df)
plt.title("Spouse and External Score")

### External Score and Target

In [ ]:
plt.figure(figsize=(16,10))
sns.swarmplot(x="target", y="external_score", data=df)
plt.title("Target and External Score")

In [ ]:
df[["external_score", "target"]].corr()

**Observations:** As we can see from the plot, there seems to be a correlation between the "external_score" of the client and whether they defaulted or not. Calculating the correlation score confirms our observations. It looks like any client with a "external_score" above 40 defaults, which means that having a lower score is better than having a high one.

In [ ]:
plt.figure(figsize=(16,10))
sns.swarmplot(x="big_city", y="external_score", data=df)
plt.title("Big City and External Score")

**Observations:** This seems to tell us that the majority of people who don't live in the big city have a "external_score" less than 40, which means that they won't default. Almost every client who lives in the big city has a "external_score" less than 40. There seems to be correlation between living in the big city and not defaulting.

In [ ]:
df.groupby(["big_city", "target"])["target"].count()

In [ ]:
plt.figure(figsize=(16,10))
sns.swarmplot(x="target", y="income_gross", data=df)
plt.title("Target and Income Gross")

In [ ]:
df[["income_gross", "target"]].corr()

**Observations:** There doesn't seem to be any strong correlation between how much a client makes and weather they default or not. Although the majority of people with a lower gross income does not default

## Data Cleaning

### Dropping columns
Removing columns that don't seem to provide much useful information when predicting loan default. These are columns that meet any of the following criteria:

1. High percentage of missing values
2. Pairwise correlation (Redundant columns)
3. Low variation in the values
4. Non informative (i.e. id)

By dropping unsuful columns, we will make it computationally easier to do better feature selection during the model building process.

In [ ]:
drop_colums = ["id", "email_domain", "contact_channel", "debt_requests_count", "housing_rent", "housing_rent", "credit_used_account", "customer_postal", "lender_id", "inquiries_count_12m", "loan_fee", "loan_interest", "hour_1", "day", "housing_base_cost"]
clean_df = df.drop(columns=drop_colums)
clean_df.head()

In [ ]:
clean_df.dropna(how="any", inplace=True)

In [ ]:
clean_df.shape

### Saving the clean data

In [ ]:
clean_df.to_pickle("clean_df.pkl")